In [1]:
import sqlite3

In [2]:
conn=sqlite3.connect('example.db')

In [3]:
conn

In [4]:
conn.execute("""CREATE TABLE IF NOT EXISTS users (id INTEGER PRIMARY KEY,
              name TEXT, 
            age INTEGER
             salary REAL not null)""") 

In [5]:
conn.execute("""CREATE TABLE IF NOT EXISTS employees (id INTEGER PRIMARY KEY,
              first_name TEXT, 
            age INTEGER
             email REAL not null)""") 

In [7]:

conn.execute("""CREATE TABLE IF NOT EXISTS customer (
                          id INTEGER PRIMARY KEY,
                          first_name TEXT NOT NULL,
                          last_name TEXT NOT NULL,
                          age INTEGER,
                          email TEXT NOT NULL,
                          phone_number TEXT,
                          address TEXT
                         )""")